# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [ ]:
#%pip install duckdb==0.6.0
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine
#%pip install duckdb-engine==0.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.2 MB/s eta 0:00:00


In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False



In [ ]:
#%sql duckdb:///:default:
%sql duckdb://

In [ ]:
#dataset_path = '/content/drive/MyDrive/DMEyF/2024/datos/'
dataset_path = '/content/DMEyF/'
dataset_file = 'competencia_01.csv'

In [ ]:
%%sql
create  table competencia_01 as
select  * from read_csv_auto("{{dataset_path + dataset_file}}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
#%%sql rollback

,Success


In [ ]:
#COMPETENCIA KAGGLE 1
#Consulta prueba
%%sql
select
    *
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
    , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
    , ntile(10) over (partition by foto_mes order by ctrx_quarter) as mov_voluntario_1
    , lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cta_saldo
    , mcuentas_saldo - lag_1_cta_saldo as delta_1_cta_saldo
    , mcuentas_saldo - lag(mcuentas_saldo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cta_saldo
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_1
    ,regr_slope(ctrx_quarter, mcuenta_corriente) over ventana_3 as ctrx_quarter_slope_1
    , Visa_status*mcuentas_saldo as iterac_1_visa_saldocta
    , cprestamos_personales*mprestamos_personales as iterac_2_cant_monto_prestamoper
    , mrentabilidad_annual*cliente_antiguedad as iterac_3_renta_antiguedad
    , if(mprestamos_personales==0,1,log10(mprestamos_personales)) as log10_1_prestamosper
    , if(mactivos_margen<=0,1,log10(mactivos_margen)) as log10_2_ganainter_antig
    , lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_1
    , lag(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_2
    , lag(Visa_mfinanciacion_limite, 2) over (partition by numero_de_cliente order by foto_mes) as mfinanciamientoVisa_menos_2
    , lag(Master_mfinanciacion_limite, 2) over (partition by numero_de_cliente order by foto_mes) as mfinanciamientoMaster_menos_2
    , lead(Master_mlimitecompra, 2) over (partition by numero_de_cliente order by foto_mes) as mlimiteMaster_mas_2
    , lag(Master_mlimitecompra, 2) over (partition by numero_de_cliente order by foto_mes) as mlimiteMaster_menos_2
    , lead(Master_mlimitecompra,1) over (partition by numero_de_cliente order by foto_mes) as mlimiteMaster_mas_1
    , lag(Master_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as mlimiteMaster_menos_1
    , ntile(4) over (partition by foto_mes order by Master_mlimitecompra) as limite_compraMaster_1
    , ntile(4) over (partition by foto_mes order by Visa_mlimitecompra) as limite_compraVisa_1
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,mrentabilidad_menos_1,mrentabilidad_menos_2,mfinanciamientoVisa_menos_2,mfinanciamientoMaster_menos_2,mlimiteMaster_mas_2,mlimiteMaster_menos_2,mlimiteMaster_mas_1,mlimiteMaster_menos_1,limite_compraMaster_1,limite_compraVisa_1
0,651518954,202103,1,0,0,50,116,2614.20,45478.82,4002.00,...,3277.53,3786.24,253368.00,50673.60,414538.20,56304.00,409391.66,394787.81,3,3
1,1535133887,202103,1,0,0,29,10,-7506.16,-2377.72,5763.84,...,2107.50,-952.81,177357.60,177357.60,414538.20,197064.00,409391.66,394787.81,3,3
2,564250855,202103,1,0,0,56,114,1253.84,28139.09,308.29,...,1308.07,2011.12,410598.67,342165.56,414538.20,380183.96,409391.66,394787.81,3,3
3,1122436210,202103,1,0,0,58,70,-5278.05,-6284.48,1310.07,...,-3793.79,-6789.80,354715.20,38005.20,415813.83,42228.00,411283.12,42228.00,3,3
4,680894196,202103,1,0,0,37,175,-1749.62,-1852.20,1594.98,...,-1433.17,-1975.66,410598.67,345213.90,415813.83,383571.00,411283.12,397163.13,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,733000802,202104,1,0,0,36,166,1180.62,10687.76,1410.80,...,1242.40,1422.62,NaN,NaN,NaN,NaN,NaN,NaN,4,4
981942,736100640,202104,1,0,0,41,137,1375.34,3763.00,682.43,...,374.67,528.31,NaN,NaN,NaN,NaN,NaN,NaN,4,4
981943,737521550,202104,1,0,0,53,165,3567.60,48173.13,1914.27,...,4455.38,2372.73,NaN,NaN,NaN,NaN,NaN,NaN,4,4
981944,739381700,202104,1,0,0,69,79,-355.32,5552.99,0.00,...,231.28,-337.24,NaN,NaN,NaN,NaN,NaN,NaN,4,4


In [ ]:
%%sql
COPY (
select
    *
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
    , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
    , ntile(10) over (partition by foto_mes order by ctrx_quarter) as mov_voluntario_1
    , lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cta_saldo
    , mcuentas_saldo - lag_1_cta_saldo as delta_1_cta_saldo
    , mcuentas_saldo - lag(mcuentas_saldo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cta_saldo
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_1
    ,regr_slope(ctrx_quarter, mcuenta_corriente) over ventana_3 as ctrx_quarter_slope_1
    , Visa_status*mcuentas_saldo as iterac_1_visa_saldocta
    , cprestamos_personales*mprestamos_personales as iterac_2_cant_monto_prestamoper
    , mrentabilidad_annual*cliente_antiguedad as iterac_3_renta_antiguedad
    , if(mprestamos_personales==0,1,log10(mprestamos_personales)) as log10_1_prestamosper
    , if(mactivos_margen<=0,1,log10(mactivos_margen)) as log10_2_ganainter_antig
    , lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_1
    , lag(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_2
    , lag(Visa_mfinanciacion_limite, 2) over (partition by numero_de_cliente order by foto_mes) as mfinanciamientoVisa_menos_2
    , lag(Master_mfinanciacion_limite, 2) over (partition by numero_de_cliente order by foto_mes) as mfinanciamientoMaster_menos_2
    , lead(Master_mlimitecompra, 2) over (partition by numero_de_cliente order by foto_mes) as mlimiteMaster_mas_2
    , lag(Master_mlimitecompra, 2) over (partition by numero_de_cliente order by foto_mes) as mlimiteMaster_menos_2
    , lead(Master_mlimitecompra,1) over (partition by numero_de_cliente order by foto_mes) as mlimiteMaster_mas_1
    , lag(Master_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as mlimiteMaster_menos_1
    , ntile(4) over (partition by foto_mes order by Master_mlimitecompra) as limite_compraMaster_1
    , ntile(4) over (partition by foto_mes order by Visa_mlimitecompra) as limite_compraVisa_1
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
)
TO '{competencia_01_fe-KAGGLE1.csv' (FORMAT CSV, HEADER TRUE);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
!zip -r "/content/competencia_01_fe-KAGGLE1.zip" "/content/competencia_01_fe-KAGGLE1.csv"


  adding: content/competencia_01_fe-KAGGLE1.csv (deflated 71%)


In [ ]:
from google.colab import files
files.download("competencia_01_fe-KAGGLE1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>